In [56]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

url = "../data/data_podcasts_title_content.csv"
df = pd.read_csv(url)

In [57]:
df['metadado'] = df['content'].apply(lambda x: x.split('\n\n')[0])

In [58]:
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b\w{1,2}\b', '', text)  
    text = re.sub(r'[^\w\s]', '', text) 
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
print(df['content'].values[0][:1000])
df['content'] = df['content'].apply(preprocess_text)
print(df['content'].values[0][:1000])

Title: COMO ATINGIR A ALTA PERFORMANCE (com Eslen Delanogare e Paulo Muzy) | Os Sócios Podcast #109
Views count: 732556
Publish date: 2022-12-09 00:00:00
Author: Podcast Completo
Description: Unknown
Link video: https://www.youtube.com/watch?v=188agVtVMpE

[Música] [Música] [Aplausos] [Música] [Música] [Música] [Música] [Música] [Música] [Aplausos] [Música] [Música] [Música] [Música] [Aplausos] [Música] [Música] [Música] [Música] [Aplausos] [Música] [Música] [Música] [Música] E aí pessoal vamos começar mais episódios Episódio número 109 estou aqui como sempre com uma luperini minha esposa host e rosto do podcast Olá pessoal sejam bem-vindos a mais um episódio e qual o tema de hoje vamos falar sobre performance para vocês né já aplicarem começar esse ano que já é quase ano que vem né exatamente bem em alta performance nós estamos chegando em 2023 de fato e em todo o começo de ano geralmente o pessoal coloca um milhão de metas que é emagrecer quer pagar as dívidas que vai começar a inves

In [60]:
df['metadata_and_content_clean'] = df['metadado'] + '\n\n' + df['content']

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer() # Important note here: there are many default parameters that do not show in this call.
X = vectorizer.fit_transform(df['metadata_and_content_clean'])

In [64]:
print(df['metadata_and_content_clean'].values[100][:1000])

Title: Entendendo o SONO | Os Sócios Podcast #77
Views count: 201810
Publish date: 2022-06-23 00:00:00
Author: Podcast Completo
Description: Unknown
Link video: https://www.youtube.com/watch?v=chYBWwliAWM

title entendendo sono sócios podcast views count publish date author podcast completo description unknown link video httpswwwyoutubecomwatchchybwwliawm concepção caio sonho aviso algo vai acontecer pessoa manda mensagem preocupada bruno sonhei coisa ruim falei cara fica tranquilo tudo sonham gente acontecer agenda transar todo mundo parte grávida fino exatamente então fiquem tranquilos quanto voltando saindo agora sonho parte sono novamente muita gente comenta chamado ciclo circadiano ilha latim cerca dia sim longo evolução ficou alinhado sol nasce gente acordando sol pondo gente começa ficar relaxado pronto período leiria noite ideal então relação ciclo circadiano hoje pessoas afeta sono porque agora sol ganhando gente fim aspectos importantes sono incrível pareça sol nasce sol põe 

In [65]:
query = "longo da evolução de ficou alinhado com isso um Quando o Sol nasce a gente tá acordando E aí quando o sol está se pondo a gente já começa a ficar mais relaxado pronto para o período Leiria né da noite de seria o ideal Então como é que tá a sua relação do ciclo circadiano hoje com as pessoas e como que isso afeta"
Q = vectorizer.transform([query])

In [66]:
R = X.dot(Q.T)

In [68]:
idxs_and_scores = sorted([[idx, x[0]]for idx, x in enumerate(R.toarray())], key=lambda x: x[1], reverse=True)

In [75]:
idxs_and_scores[:10]

[[100, 0.33511509723043315],
 [99, 0.18204335602186889],
 [114, 0.12657133555836758],
 [106, 0.11385720060447238],
 [108, 0.11296138667832055],
 [115, 0.11222276653629554],
 [222, 0.10372728520760133],
 [221, 0.10108558626415878],
 [169, 0.09456714480021482],
 [168, 0.09402098134323858]]

In [79]:
# Get score with np.argsort
scores = np.array(R.toarray()).flatten()
idxs = np.argsort(scores)[::-1]
idxs_and_scores = np.array([[idx, scores[idx]] for idx in idxs])

In [85]:
idxs_and_scores[:10, 0].astype(int)

array([100,  99, 114, 106, 108, 115, 222, 221, 169, 168])

In [77]:
idxs = np.array(idxs_and_scores[:10])[:, 0].astype(int)

In [78]:
df.iloc[idxs]

,titles,content,metadado,metadata_and_content_clean
100,Entendendo o SONO | Os Sócios Podcast #77,title entendendo sono sócios podcast views cou...,Title: Entendendo o SONO | Os Sócios Podcast #...,Title: Entendendo o SONO | Os Sócios Podcast #...
99,Entendendo o SONO | Os Sócios Podcast #77,title entendendo sono sócios podcast views cou...,Title: Entendendo o SONO | Os Sócios Podcast #...,Title: Entendendo o SONO | Os Sócios Podcast #...
114,Entendendo o SONO | Os Sócios Podcast #77,title entendendo sono sócios podcast views cou...,Title: Entendendo o SONO | Os Sócios Podcast #...,Title: Entendendo o SONO | Os Sócios Podcast #...
106,Entendendo o SONO | Os Sócios Podcast #77,title entendendo sono sócios podcast views cou...,Title: Entendendo o SONO | Os Sócios Podcast #...,Title: Entendendo o SONO | Os Sócios Podcast #...
108,Entendendo o SONO | Os Sócios Podcast #77,title entendendo sono sócios podcast views cou...,Title: Entendendo o SONO | Os Sócios Podcast #...,Title: Entendendo o SONO | Os Sócios Podcast #...
115,Entendendo o SONO | Os Sócios Podcast #77,title entendendo sono sócios podcast views cou...,Title: Entendendo o SONO | Os Sócios Podcast #...,Title: Entendendo o SONO | Os Sócios Podcast #...
222,Puravida: Vitamina D e Pés no Solo | Os Sócios...,title puravida vitamina pés solo sócios podcas...,Title: Puravida: Vitamina D e Pés no Solo | Os...,Title: Puravida: Vitamina D e Pés no Solo | Os...
221,Puravida: Vitamina D e Pés no Solo | Os Sócios...,title puravida vitamina pés solo sócios podcas...,Title: Puravida: Vitamina D e Pés no Solo | Os...,Title: Puravida: Vitamina D e Pés no Solo | Os...
169,COMO FUNCIONA O CÉREBRO | Os Sócios Podcast #43,title funciona cérebro sócios podcast views co...,Title: COMO FUNCIONA O CÉREBRO | Os Sócios Pod...,Title: COMO FUNCIONA O CÉREBRO | Os Sócios Pod...
168,COMO FUNCIONA O CÉREBRO | Os Sócios Podcast #43,title funciona cérebro sócios podcast views co...,Title: COMO FUNCIONA O CÉREBRO | Os Sócios Pod...,Title: COMO FUNCIONA O CÉREBRO | Os Sócios Pod...


In [143]:
import pickle
from pydantic import BaseModel

nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

class Output(BaseModel):
    title: str
    content: str
    relevance: float

class Retriever:
    def __init__(self, path_csv, path_model):
        self.df = pd.read_csv(path_csv)
        self.df['metadado'] = self.df['content'].apply(lambda x: x.split('\n\n')[0])
        self.df['content'] = self.df['content'].apply(self.preprocess_text)
        self.df['metadata_and_content_clean'] = self.df['metadado'] + '\n\n' + self.df['content']
        try:
            with open(path_model, 'rb') as f_in:
                self.vectorizer = pickle.load(f_in)
            self.X = self.vectorizer.transform(self.df['metadata_and_content_clean'])    
            
        except:
            self.vectorizer = TfidfVectorizer()
            self.X = self.vectorizer.fit_transform(self.df['metadata_and_content_clean'])
            
        
    def invoke(self, query, k=3):
        Q = vectorizer.transform([query])
        R = self.X.dot(Q.T)
        
        # Get score with np.argsort
        scores = np.array(R.toarray()).flatten()
        idxs = np.argsort(scores)[::-1]
        idxs_and_scores = np.array([[idx, scores[idx]] for idx in idxs])
        
        retrieved = np.array(idxs_and_scores[:k])
        idxs = np.array(retrieved)[:, 0].astype(int)
        output_title = self.df.iloc[idxs][['titles']].values
        output_content = self.df.iloc[idxs][['metadata_and_content_clean']].values
        output_relevance = retrieved[:, 1].reshape(-1, 1)
        output = np.concatenate([output_title, output_content, output_relevance], axis=1)
        return output
    
    def query(self, query, k=3):
        output = self.invoke(query, k)
        output = [Output(title=x[0], content=x[1], relevance=x[2]) for x in output]
        return output
    
    def preprocess_text(self, text):
        text = text.lower()
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'\b\w{1,2}\b', '', text)  
        text = re.sub(r'[^\w\s]', '', text) 
        text = ' '.join(word for word in text.split() if word not in stop_words)
        return text
    
    def save_model(self, filename='tfidf_model.pkl'):
        # Salvar o modelo TF-IDF em um arquivo .pkl
        with open(filename, 'wb') as f_out:
            pickle.dump(self.vectorizer, f_out)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [144]:
retriever = Retriever(path_csv="../data/data_podcasts_title_content.csv", path_model='../models/tfidf_model.pkl')

In [146]:
query = "longo da evolução de ficou alinhado com isso um Quando o Sol nasce a gente tá acordando E aí quando o sol está se pondo a gente já começa a ficar mais relaxado pronto para o período Leiria né da noite de seria o ideal Então como é que tá a sua relação do ciclo circadiano hoje com as pessoas e como que isso afeta"

retriever.query(query, k=5)

[Output(title='Entendendo o SONO | Os Sócios Podcast #77', content='Title: Entendendo o SONO | Os Sócios Podcast #77\nViews count: 201810\nPublish date: 2022-06-23 00:00:00\nAuthor: Podcast Completo\nDescription: Unknown\nLink video: https://www.youtube.com/watch?v=chYBWwliAWM\n\ntitle entendendo sono sócios podcast views count publish date author podcast completo description unknown link video httpswwwyoutubecomwatchchybwwliawm concepção caio sonho aviso algo vai acontecer pessoa manda mensagem preocupada bruno sonhei coisa ruim falei cara fica tranquilo tudo sonham gente acontecer agenda transar todo mundo parte grávida fino exatamente então fiquem tranquilos quanto voltando saindo agora sonho parte sono novamente muita gente comenta chamado ciclo circadiano ilha latim cerca dia sim longo evolução ficou alinhado sol nasce gente acordando sol pondo gente começa ficar relaxado pronto período leiria noite ideal então relação ciclo circadiano hoje pessoas afeta sono porque agora sol ganh

In [147]:
retriever.save_model(filename='../models/tfidf_model.pkl')